# Bart vs. Homer 

## Imports

In [1]:
%matplotlib inline
import platform

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

import skimage

import sklearn
from sklearn.preprocessing import scale

import numpy as np

import tensorflow
from tensorflow.keras.layers import (BatchNormalization, Conv2D, MaxPooling2D,
                                    GlobalAveragePooling2D, Activation)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, InputLayer
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

from livelossplot import PlotLossesKeras

Using TensorFlow backend.


In [2]:
print "Python version:", platform.python_version()
print "Matplotlib version:", matplotlib.__version__
print "SKImage version:", skimage.__version__
print "SKLearn version:", sklearn.__version__
print "Numpy version:", np.__version__
print "Tensorflow version:", tensorflow.__version__

Python version: 2.7.15rc1
Matplotlib version: 2.2.3
SKImage version: 0.14.1
SKLearn version: 0.20.0
Numpy version: 1.15.2
Tensorflow version: 1.11.0


## Utils functions 

In [ ]:
label2character = dict(enumerate(["Bart", "Homer", "Background"]))
character2label = {character: label for label, character in label2character.items()}

In [ ]:
def get_data_and_labels(directory, shuffle=True):
    gen = image.ImageDataGenerator()
    batches = gen.flow_from_directory(directory, target_size=(360, 640), batch_size=1, shuffle=shuffle)

    imgs = []
    labels = []
    for i in range(batches.n):
        img, label = batches.next()
        imgs.append(img)
        labels.append(label)

    data = np.concatenate(imgs)
    labels = np.concatenate(labels)
    
    return data, labels

In [ ]:
def display_image_and_heatmap(images, heatmaps, predictions, number):
    pred_bart, pred_homer = predictions[number]
    label = np.argmax(predictions[number])
    print "Predicted label:", label2character[label]
    print "Bart confidence:", round(pred_bart * 100, 2), "% - Homer confidence:", round(pred_homer * 100, 2), "%"
    
    fig = plt.figure(figsize=(18, 18))
    img = images[number].astype(np.uint8)

    for character in ("Bart", "Homer"):
        label = character2label[character]
        fig.add_subplot(2, 1, label + 1, title="Heatmap for " + character + " recognition")
        heat_map_low_res = np.moveaxis(heatmaps[number], 2, 0)[label]
        heat_map = skimage.transform.resize(heat_map_low_res, (360, 640), mode="reflect", anti_aliasing=True)
        plt.imshow(img)
        plt.imshow(heat_map, cmap="seismic", alpha=0.5, vmax=heatmaps[number].max())
        plt.colorbar()

In [ ]:
def display_image_and_heatmap_with_background(images, heatmaps, predictions, number):
    pred_bart, pred_homer, pred_background = predictions[number]
    label = np.argmax(predictions[number])
    print "Predicted label:", label2character[label]
    print "Bart confidence:", round(pred_bart * 100, 2), "% - Homer confidence:", round(pred_homer * 100, 2), "% - Background confidence:", round(pred_background * 100, 2), "%" 
    
    fig = plt.figure(figsize=(18, 18))
    img = images[number].astype(np.uint8)

    for character in ("Bart", "Homer", "Background"):
        label = character2label[character]
        fig.add_subplot(3, 1, label + 1, title="Heatmap for " + character + " recognition")
        heat_map_low_res = np.moveaxis(heatmaps[number], 2, 0)[label]
        heat_map = skimage.transform.resize(heat_map_low_res, (360, 640), mode="reflect", anti_aliasing=True)
        plt.imshow(img)
        plt.imshow(heat_map, cmap="seismic", alpha=0.5, vmax=heatmaps[number].max())
        plt.colorbar()

In [ ]:
def create_model_and_fit(training_directory, validation_directory, nb_classes, nb_epochs,
                         with_background=False, record_input_global_average_pooling=False):
    train_data, train_labels = get_data_and_labels(training_directory)
    val_data, val_labels = get_data_and_labels(validation_directory, False)
    
    vgg16_bottom = VGG16(include_top=False, input_shape=(360, 640, 3), weights='imagenet')
    model_bottom = Sequential(VGG16(include_top=False, input_shape=(360, 640, 3), weights='imagenet').layers[:-1])
    
    print "Predict output of non trainable layers: Training set"
    post_model_bottom_train_features = model_bottom.predict(train_data, batch_size=1, verbose=1)
    
    print "Predict output of non trainable layers: Validation set"
    post_model_bottom_val_features = model_bottom.predict(val_data, batch_size=1, verbose=1)
    
    model_top = Sequential([
                Conv2D(nb_classes, (3, 3), activation='relu', padding='same', name="conv_last"),
                GlobalAveragePooling2D(name="global_average_pooling"),
                Activation("softmax", name="softmax")
            ])
    
    model_top.compile(loss=categorical_crossentropy, optimizer=Adam(1e-3), metrics=[categorical_accuracy])

    print "Fit the last convolutional layer"
    if not with_background:
        model_top.fit(post_model_bottom_train_features, train_labels,
                      validation_data=(post_model_bottom_val_features, val_labels),
                      epochs=nb_epochs, verbose=0, callbacks=[PlotLossesKeras()], batch_size=1)
    else:
        model_top.fit(post_model_bottom_train_features, train_labels,
                      validation_data=(post_model_bottom_val_features, val_labels),
                      epochs=nb_epochs, verbose=0, callbacks=[PlotLossesKeras()], batch_size=1,
                      class_weight = {0: 1, 1: 1, 2: 15})        
    
    inp = x = Input(shape=(360, 640, 3))

    for layer in model_bottom.layers + model_top.layers:
        if layer.name == 'global_average_pooling':
            conv_last = x
        x = layer(x)
    
    model = Model([inp], [x, conv_last])    
    
    return model, train_data, val_data

## Train In Front Of House

### Train the model

In [ ]:
model, train_data, val_data = create_model_and_fit("InFrontOfHouse/Train", "InFrontOfHouse/Valid", 2, 20)

### Predict labels and last convolutional layer

In [ ]:
val_predictions, val_last_conv = model.predict(val_data, verbose=1, batch_size=1)

### Display validation data

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 0)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 1)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 2)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 3)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 4)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 5)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 6)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 7)

### Predict labels and last convolutional layer on an other background

In [ ]:
in_front_of_nuclear_data, _ = get_data_and_labels("InFrontOfNuclear")
in_front_of_nuclear_predictions, in_front_of_nuclear_last_conv = model.predict(in_front_of_nuclear_data, verbose=1, batch_size=3)

### Display data on an other background

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 0)

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 1)

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 2)

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 3)

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 4)

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 5)

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 6)

In [ ]:
display_image_and_heatmap(in_front_of_nuclear_data, in_front_of_nuclear_last_conv, in_front_of_nuclear_predictions, 7)

## Train In Front Of Several backgrounds

### Train the model

In [ ]:
model, train_data, val_data = create_model_and_fit("SeveralBackgrounds/Train", "SeveralBackgrounds/Valid", 2, 30)

### Predict labels and last convolutional layer

In [ ]:
val_predictions, val_last_conv = model.predict(val_data, verbose=1, batch_size=3)

### Display validation data

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 0)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 1)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 2)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 3)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 4)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 5)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 6)

In [ ]:
display_image_and_heatmap(val_data, val_last_conv, val_predictions, 7)

### Predict labels and last convolutional layer on inverted backgrounds

In [ ]:
inverted_backgrounds_data, _ = get_data_and_labels("InvertedBackgrounds")
inverted_backgrounds_predictions, inverted_backgrounds_last_conv = model.predict(inverted_backgrounds_data, verbose=1, batch_size=3)

### Display data on an inverted backgrounds

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 0)

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 1)

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 2)

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 3)

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 4)

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 5)

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 6)

In [ ]:
display_image_and_heatmap(inverted_backgrounds_data, inverted_backgrounds_last_conv, inverted_backgrounds_predictions, 7)

### Predict labels and last convolutional layer for background only

In [ ]:
background_only_data, _ = get_data_and_labels("BackgroundOnly")
background_only_predictions, background_only_last_conv = model.predict(background_only_data, verbose=1, batch_size=3)

### Display data for background only

In [ ]:
display_image_and_heatmap(background_only_data, background_only_last_conv, background_only_predictions, 0)

In [ ]:
display_image_and_heatmap(background_only_data, background_only_last_conv, background_only_predictions, 1)

## Train With Backgrounds as Class

### Train the model

In [ ]:
model, train_data, val_data = create_model_and_fit("WithBackgroundsAsClass/Train", "WithBackgroundsAsClass/Valid", 3,  40, True)

### Predict labels and last convolutional layer

In [ ]:
val_predictions, val_last_conv = model.predict(val_data, verbose=1, batch_size=3)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 0)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 1)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 2)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 3)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 4)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 5)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 6)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 7)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 8)

In [ ]:
display_image_and_heatmap_with_background(val_data, val_last_conv, val_predictions, 9)